In [1]:
import numpy as np
import importlib
import nbimporter

import vapour_pressure as vp

Importing Jupyter notebook from vapour_pressure.ipynb


### 物性値を取得するクラス  
各物性値ファイルに存在する物性値を元の形式のまま取得してくるクラス。  

#### 実験から熱水分物性値として考えられるもの  
【参考】建築材料の熱・空気・湿気物性値，日本建築学会，丸善株式会社，2001など

#### 熱物性値  
$\dot q$：単位時間当たりの熱流量の密度[W/m2]  
$\rho_0$：密度[kg/m3]  
$c_0$：比熱[J/kg K]  
$c_0\rho_0$：容積熱容量[J/m3 K]  

$\lambda$：熱伝導率[W/mK]  
定義：$\dot q = -\lambda・grad\space T$  

$R$：熱抵抗[Km2/W]  

#### 水分物性値  
$\dot m_v$：単位時間当たりの水蒸気流量の密度[kg/m2 s]  
$\dot m_m$：単位時間当たりの水分流量の密度[kg/m2 s]  
$\phi$：含水率[m3/m3]  
他標記として$w$[kg/m3]、$u$[kg/kg]など  
$S$：飽和度[-]  
#### 気相水分伝導率  
$\delta_p$：透湿率[kg/m Pa s]　文中記号DP  
定義：$m_v = -\delta_p・grad\space p_v$  

$Z_p$：湿気抵抗[m2Pa s/kg]  
$\delta_v$：水蒸気の拡散係数[m2/s]  
定義：$\dot m_v = -\delta_v・grad\space \rho_v$

$k_T$：温度勾配水分伝導率[kg/m K s]  
定義：$\dot m_m = -k_T・grad\space T$  

$D_T$：温度勾配水分拡散係数[m2/K s]  
定義：$\dot m_m = -\rho_0 D_T・grad\space T$  

$\lambda^{'}_{\mu g}$：水分化学ポテンシャル勾配に対する気相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu g}・grad\space \mu$

$\lambda^{'}_{Tg}$：温度勾配に対する気相水分伝導率[kg/ms K]  
定義：$\dot m_v = -\lambda^{'}_{Tg}・grad\space \mu$

#### 液相水分伝導率  
$k_m$：水分伝導率[kg/m Pa s]  
定義：$\dot m_m = -k_m・grad\space s$

$D_w$：水分拡散係数[m2/s]  
定義：$\dot m_m = -\rho_0 D_w・grad\space u$  

$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に対する液相水分伝導率[kg/ms (J/kg)]  
定義：$\dot m_v = -\lambda^{'}_{\mu l}・grad\space \mu$

#### 空気の拡散係数  
$k_a$：透気率[m2/Ks]  


※新しい基準で物性値を測定した場合、本クラスに書き加えること


### 取得した物性値を流量計算に適した形に換算するクラス

#### 物性名からインポートすべきモジュール名を検索する

In [2]:
def getModuleName( name ):
    try:
        return {
            'BentheimerSandstone': 'bentheimer_sandstone',        
        }[name]
    except:
        raise ValueError('物性リストに登録されていない物性名が指定されました。')    

#### インポートすべきモジュール名をもとに物性値計算を定義した外部ファイルをインポートする

In [3]:
def importModule( moduleName ):
    try:
        return importlib.import_module( moduleName )
    except:
        raise ImportError('物性リストに登録されている外部モジュールが見つかりません。')

#### 水の気体定数

In [4]:
def Rv():
    return 8316.0/18.016

#### 水の密度

In [5]:
def row():
    return 1000.0

#### 重力加速度

In [6]:
def grav():
    return 9.806650

In [7]:
def getDPDM( temp, miu ):
    return vp.Pvs( temp ) * np.exp( miu / Rv() / temp ) / Rv() / temp

In [8]:
def getDPDT( temp, miu ):
    return np.exp( miu / Rv() / temp ) * ( vp.DPvs( temp ) - vp.Pvs( temp ) * miu / Rv() / ( temp**2 ) )

In [9]:
class MaterialKernel():
    # このクラスは Material そのものを表すのではなくて、あくまで CROW 等の物性を計算するアルゴリズムを提供するだけなので、
    # 名前に Kernel とつけました。
    
    def __init__( self, name ):
        
        # 物性名称からインポートすべきモジュール名称を検索する。
        moduleName = getModuleName( name )
        
        # インポートすべきモジュールをインポートする。モジュールが存在しなければエラー。
        module = importModule( moduleName )
        
        if ( 'getCRow' in dir( module ) ):
            self.getCRow = module.getCRow
        else:
            # どちらも存在しない場合は関数を追加しない。
            pass
        # 以下、同じ。
        
        if( 'getLAM' in dir( module ) ):
            self.getLAM = module.getLAM
        else:
            pass

        if( 'getPhi' in dir( module ) ):
            self.getPhi = module.getPhi
        else:
            pass

        if( 'getDw' in dir( module ) ):
            self.getDw = module.getDw
        else:
            pass

        if( 'getDP' in dir( module ) ):
            self.getDP = module.getDP
        else:
            pass

        if( 'getDPhi' in dir( module ) ):
            self.getDPhi = module.getDPhi
        else:
            pass

        if( 'getMiuByPhi' in dir( module ) ):
            self.getMiuByPhi = module.getMiuByPhi
        else:
            pass
        
        if( 'getLDMG' in dir( module ) ):
            self.getLDMG = module.getLDMG
        elif( 'getDP' in dir( module ) ):
            # 本クラスに DP を計算する式が存在すれば、LDMG の計算を DP を用いて行う。
            # getDPByRH が定義されていれば、getDPByMiu も定義されていれはずなので、ここでは getDPByMiu 関数のみ検索する。
            self.getLDMG = lambda c : getDPDM( c.temp, c.miu ) * self.getDP( c )
        else:
            pass
        
        if( 'getLDTG' in dir(module) ):
            self.getLDTG = module.getLDTG
        elif( 'getDP' in dir( module ) ):
            # 本クラスに DP を計算する式が存在すれば、LDTG の計算を DP を用いて行う。
            # getDPByRH が定義されていれば、getDPByMiu も定義されていれはずなので、ここでは getDPByMiu 関数のみ検索する。
            self.getLDTG = lambda c : getDPDT( c.temp, c.miu ) * self.getDP( c )
        else:
            pass

        if( 'getLDML' in dir(module) ):
            self.getLDML = module.getLDML
        elif( 'getDw' in dir( module ) ):
            # 本クラスに Dw を計算する式が存在すれば、LDML の計算を Dw を用いて行う。
            # getDwByRH が定義されていれば、getDwByMiu も定義されていれはずなので、ここでは getDwByMiu 関数のみ検索する。
            self.getLDML = lambda c : row() / grav() * self.getDw( c )
        else:
            pass        

In [10]:
m = MaterialKernel('BentheimerSandstone')

Importing Jupyter notebook from bentheimer_sandstone.ipynb
Importing Jupyter notebook from van_genuchten.ipynb


In [11]:
# 無くても良い（他のモデュールに実装されている）
class HydrothermalInfo():
    def temp( self, initial = 0.0 ):
        self.temp = initial 
    def RH( self, initial = 0.0 ):
        self.rh = initial  
    def miu( self, initial = 0.0 ):
        self.miu = initial  
    def pv( self, initial = 0.0 ):
        self.pv = initial 
    def phi( self, initial = 0.0 ):
        self.phi = initial 

In [12]:
c = HydrothermalInfo()
c.temp( 293.15 ) 
c.miu( -10.0 ) 
c.phi( 0.22 )

In [13]:
m.getPhi( c )

0.16098357394272111

In [14]:
m.getLDML( c )

1.3935469734779378e-06